In [20]:
import os
os.chdir('/mnt/ialabnas/homes/fidelrio/systematic-text-representations/')

import json
from pathlib import Path
import random
import pprint

import matplotlib
import matplotlib.pyplot as plt

import os

import torch

from config import load_config
from data import build_datasets, build_loader
from data import (CollatorForMaskedLanguageModeling,
                  CollatorForMaskedSelectedTokens,
                  CollatorForMaskedRandomSelectedTokens,
                  IdentityCollator)
from data import ALL_POSSIBLE_COLORS
from model import MultimodalModel, MultimodalPretrainingModel
from utils import load_checkpoint
from lightning import Trainer, seed_everything
from tqdm.auto import tqdm, trange

from PIL import Image

from torch.utils.data import DataLoader, Subset
from torch.nn.functional import softmax


pp = pprint.PrettyPrinter(indent=2)

In [2]:
def scene_tensor_to_txt(tensor):
    return ' '.join([processor.inv_vocabulary[t] if t != -100 else '[NONE]' for t in tensor.tolist()])

def print_scene_tensor(tensor):
    scene_text = scene_tensor_to_txt(tensor)
    print(scene_text.replace('[PAD]', '').replace('[SEP]','\n     '))
    
def print_parallel(tensor0, tensor1, tensor2, confidences, titles):
    ttl0, ttl1, ttl2 = titles
    print(f'{ttl0:6.6s} {ttl1:6.6s} {ttl2:6.6s}')
    for t0, t1, t2, conf in zip(
            tensor0.tolist(), tensor1.tolist(), tensor2.tolist(), confidences.tolist()):
        w0 = processor.inv_vocabulary[t0]
        w1 = processor.inv_vocabulary[t1]
        w2 = processor.inv_vocabulary[t2]
        
        if w0 == '[SEP]':
            print()
            continue
        if w0 == '[PAD]':
            break
        
        print_txt = f'{w0:6.6s} {w1:6.6s} {w2:6.6s} ({conf:.4f})'
        if w0 != w2:
            print_txt = bold(print_txt)
            

        print(print_txt)
        
def bold(text):
    return ("\033[1m" + text + "\033[0m")

In [3]:
device = torch.device('cuda')

n_colors = '8c'
epoch = None
# exp_name = f'mmlm--n_colors={n_colors}c--mlm_probability=0.15'
exp_name = f'mmlm--n_colors={n_colors}--mlm_probability=0.15'

checkpoint = load_checkpoint(exp_name, epoch=epoch)
print('Epoch:', checkpoint['epoch'])

Epoch: 999


In [4]:
config = load_config(exp_name)

config.vocabulary_path = config.vocabulary_path.replace('/workspace/' ,'/workspace1/')
# config.vocabulary_path = f'/mnt/ialabnas/homes/fidelrio/clevr-dataset-gen/output/vocab-{n_colors}.txt'
config.base_path = config.base_path.replace('/workspace/' ,'/workspace1/')
# config.base_path = f'/mnt/ialabnas/homes/fidelrio/clevr-dataset-gen/output/multicolored-v2/{n_colors}/'

Loading mmlm--n_colors=8c--mlm_probability=0.15 last checkpoint config from outputs/mmlm--n_colors=8c--mlm_probability=0.15/last.ckpt
Add new arg: trainset_subset = 1.0
Add new arg: permute_pixels = False
Add new arg: color_jitter = False
Add new arg: color_jitter_brightness = 0.0
Add new arg: color_jitter_hue = 0.0
Add new arg: color_jitter_saturation = 0.0
Add new arg: color_jitter_contrast = 0.0
Add new arg: aug_zero_color = False


In [5]:
config.episodic_training = True

In [7]:
!ls /mnt/ialabnas/homes/fidelrio/clevr-dataset-gen/output/multicolored-v2/

 125c		  27e1_1	 '64c_3_ep=256'   64e2_10	'8c_1_pb=1.0'
 125e1_1	  27e1_10	 '64c_3_ep=64'	  64e2_2	 8c2
 125e1_2	  27e1_2	 '64c_3_pb=0.5'   64e2_3	'8c_2_pb=0.5'
 125e1_3	  27e1_3	 '64c_3_pb=1.0'   64e2_4	'8c_2_pb=1.0'
 125e1_4	  27e1_4	 '64c_3_pb=256'   64e2_5	 8c3
 125e1_5	  27e1_5	 '64c_4_ep=256'   64e2_6	'8c_3_pb=0.5'
 216c		  27e1_6	 '64c_4_ep=64'	  64e2_7	'8c_3_pb=1.0'
'216c_1_ep=256'   27e1_7	 '64c_4_pb=0.5'   64e2_8	 8c4
'216c_1_ep=64'	  27e1_8	 '64c_4_pb=1.0'   64e2_9	'8c_4_pb=0.5'
 216c2		  27e1_9	 '64c_4_pb=256'   64e3_1	'8c_4_pb=1.0'
'216c_2_ep=256'   27e2_1	 '64c_5_ep=256'   64e3_10	 8c5
'216c_2_ep=64'	  27e2_2	 '64c_5_ep=64'	  64e3_2	'8c_5_pb=0.5'
 216c3		  27e2_3	 '64c_5_pb=0.5'   64e3_3	'8c_5_pb=1.0'
'216c_3_ep=256'   27e2_4	 '64c_5_pb=1.0'   64e3_4	 8d1
'216c_3_ep=64'	  27e2_5	 '64c_5_pb=256'   64e3_5	 8d2
 216c4		  64c		  64e1_1	  64e3_6	 8d3
'216c_4_ep=256'  '64c_1_ep=256'   64e1_10	  64e3_7	 8e1_1
'216c_4_ep=64'	 '64c_1_ep=64'	  64e1_2	  64e3_8	 8e1_2
 216c5		 '64c_1

In [8]:
# pp.pprint(vars(config))

In [9]:
train_dataset, test_dataset, systematic_dataset, common_systematic_dataset = build_datasets(config)
config.pad_idx = train_dataset.pad_idx

Building CLEVRMultimodalSplit


In [10]:
model = MultimodalModel(config)
training_model = MultimodalPretrainingModel(model, config)
training_model.load_state_dict(checkpoint['state_dict'])

/home/fidelrio/.pyenv/versions/systematicity/lib/python3.7/site-packages/lightning/pytorch/utilities/parsing.py:270: UserWarning: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
  f"Attribute {k!r} is an instance of `nn.Module` and is already saved during checkpointing."


<All keys matched successfully>

In [11]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

5572448

In [12]:
processor = test_dataset.processor

In [13]:
relation_tokens = sorted([processor.vocabulary[w] for w in ['left', 'right', 'behind', 'front']])
color_tokens = sorted(
    [processor.vocabulary[w] for w in ALL_POSSIBLE_COLORS if w in processor.vocabulary])
shapes_tokens = sorted([processor.vocabulary[w] for w in ['cylinder', 'sphere', 'cube']])
materials_tokens = sorted([processor.vocabulary[w] for w in ['metal', 'rubber']])
size_tokens = sorted([processor.vocabulary[w] for w in ['small', 'large']])

In [14]:
# [w for w in ALL_POSSIBLE_COLORS if w in processor.vocabulary]

In [15]:
# set([w for w in ALL_POSSIBLE_COLORS if '#' in w]) & set([w for w in processor.vocabulary.keys() if '#' in w])  

In [16]:
!ls /workspace1/fidelrio/CLEVR_CoGenT_v1.0/colored-v2/216c/

CoGenT_A.json  CoGenT_B.json  images  properties.json  scenes  vocab.txt


In [17]:
# from torch.utils.data import default_collate

# class CollatorForMaskedSelectedTokens:
#     def __init__(self, config, processor, tokens, dont_mask_spheres=False):
#         self.config = config
#         self.token_to_mask_idxs = torch.tensor(tokens).long()
#         self.special_token_idxs = torch.tensor(processor.special_token_idxs).long()
#         self.mask_token_idx = processor.mask_token_idx
#         self.image_patch_sizes = config.patch_height, config.patch_width
#         self.dont_mask_spheres = dont_mask_spheres
#         self.sphere_token_idx = processor.vocabulary['sphere']

#     def __call__(self, batch):
#         images, scenes = default_collate(batch)
#         scenes, scenes_labels = self.build_targets(scenes)
#         images_labels = self.build_null_image_targets(images)
#         labels = torch.cat((images_labels, scenes_labels), dim=1)
#         return images, scenes, labels

#     def build_null_image_targets(self, images):
#         b, *_ = images.shape
#         n_patches = self.config.n_patches
#         return torch.full((b, n_patches), -100)

#     def build_targets(self, inputs):
#         labels = inputs.clone()
#         masked_indices = torch.isin(labels, self.token_to_mask_idxs)
#         if self.dont_mask_spheres:
#             sphere_token_mask = inputs == self.sphere_token_idx
#             sphere_mask = (torch.roll(sphere_token_mask, shifts=-3, dims=1) |
#                            torch.roll(sphere_token_mask, shifts=-2, dims=1) |
#                            torch.roll(sphere_token_mask, shifts=-1, dims=1) |
#                            sphere_token_mask)
#             masked_indices = masked_indices & ~sphere_mask
            
#         labels[~masked_indices] = -100
#         inputs[masked_indices] = self.mask_token_idx

#         return inputs, labels


# class CollatorForMaskedRandomSelectedTokens:
#     def __init__(self, config, processor, tokens, p, dont_mask_spheres=False):
#         self.config = config
#         self.token_to_mask_idxs = torch.tensor(tokens).long()
#         self.special_token_idxs = torch.tensor(processor.special_token_idxs).long()
#         self.mask_token_idx = processor.mask_token_idx
#         self.image_patch_sizes = config.patch_height, config.patch_width
#         self.p = p
#         self.dont_mask_spheres = dont_mask_spheres
#         self.sphere_token_idx = processor.vocabulary['sphere']

#     def __call__(self, batch):
#         images, scenes = default_collate(batch)
#         scenes, scenes_labels = self.build_targets(scenes)
#         images_labels = self.build_null_image_targets(images)
#         labels = torch.cat((images_labels, scenes_labels), dim=1)
#         return images, scenes, labels

#     def build_null_image_targets(self, images):
#         b, *_ = images.shape
#         n_patches = self.config.n_patches
#         return torch.full((b, n_patches), -100)

#     def build_targets(self, inputs):
#         labels = inputs.clone()
#         masked_indices = torch.isin(labels, self.token_to_mask_idxs)
#         is_selected = torch.bernoulli(torch.full_like(labels, self.p, dtype=torch.float)).bool()
#         masked_indices = masked_indices & is_selected
#         if self.dont_mask_spheres:
#             sphere_token_mask = inputs == self.sphere_token_idx
#             sphere_mask = (torch.roll(sphere_token_mask, shifts=-3, dims=1) |
#                            torch.roll(sphere_token_mask, shifts=-2, dims=1) |
#                            torch.roll(sphere_token_mask, shifts=-1, dims=1) |
#                            sphere_token_mask)
#             masked_indices = masked_indices & ~sphere_mask
            
#         labels[~masked_indices] = -100
#         inputs[masked_indices] = self.mask_token_idx

#         return inputs, labels


In [18]:
# collator = CollatorForMaskedLanguageModeling(config, processor)
# collator = CollatorForMaskedSelectedTokens(config, processor, tokens=color_tokens, dont_mask_spheres=True)
collator = CollatorForMaskedRandomSelectedTokens(config, processor, tokens=color_tokens, p=0.8, dont_mask_spheres=True)
# collator = IdentityCollator(config, processor)

In [21]:
train_loader = build_loader(train_dataset, config)

KeyError: 'episode_id'

In [ ]:
# sample_idx = 1
sample_idx = random.randint(0, len(test_dataset))
image, scene = test_dataset.retrieve_raw(sample_idx)
image_tensor, scene_tensor = test_dataset[sample_idx]

collated_images, collated_scenes, collated_labels = collator([(image_tensor, scene_tensor)])

print(sample_idx)

In [ ]:
print_scene_tensor(scene_tensor)
print()
print_scene_tensor(collated_scenes[0])

image

In [ ]:
# print_scene_tensor(scene_tensor)
# print()
# print_scene_tensor(collated_scenes[0])
# print()
# print_scene_tensor(collated_labels[0,-50:])

In [ ]:
from collections import Counter

to_properties = lambda object_: (object_['size'], object_['material'], object_['color'], object_['shape'])

def to_object_sets(objects):
    c = Counter()
    object_set = set()
    for object_ in objects:
        object_properties = to_properties(object_)
        id_ = c[object_properties]
        c[object_properties] += 1
        object_set.add(tuple([id_, *object_properties]))

    return object_set

In [ ]:
sample_idx = random.randint(0, len(test_dataset))
image, scene = test_dataset.retrieve_raw(sample_idx)
image

In [ ]:
# from torchvision.transforms import v2
from torchvision import transforms
jitter = transforms.ColorJitter(brightness=.0, hue=.5, contrast=.0, saturation=.0)

jitter(image)

In [ ]:
images, scenes = train_dataset[sample_idx]
images, scenes = images.unsqueeze(0), scenes.unsqueeze(0)

In [ ]:
sphere_token_idx = processor.vocabulary['sphere']
sphere_token_mask = scenes == sphere_token_idx
sphere_mask = (torch.roll(sphere_token_mask, shifts=-3, dims=1) |
               torch.roll(sphere_token_mask, shifts=-2, dims=1) |
               torch.roll(sphere_token_mask, shifts=-1, dims=1) |
               sphere_token_mask)

In [ ]:
# import numpy as np
# dataset = train_dataset

# with open(config.base_path + f'/CoGenT_A.json') as fp:
#     color_dist = json.load(fp)

# common_colors = set(color_dist['cube']) & set(color_dist['cylinder'])
# cmn_colors_in_image = np.array([
#     np.mean([o['color'] in common_colors for o in scene['objects']])
#     for scene in dataset.scenes
# ])

# CUTS = [0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,-0.0]

# subsets = []
# for cut in CUTS:
#     indices = np.argwhere(cmn_colors_in_image >= cut)[:,0].tolist()
#     subsets.append(Subset(dataset, indices))

In [ ]:
# [len(s) for s in subsets]

In [ ]:
output_logits = model(collated_images, collated_scenes)

confidences = softmax(output_logits, dim=-1).max(dim=-1).values
predictions = output_logits.argmax(dim=-1)

In [ ]:
print_parallel(
    scene_tensor,
    collated_scenes[0],
    predictions[0][-config.max_scene_size:],
    confidences[0][-config.max_scene_size:],
    titles=['gd_tth', 'input', 'output']
)

In [ ]:
trainer = Trainer(max_epochs=config.max_epochs,
                  accelerator="gpu",
                  devices=torch.cuda.device_count()
)

In [ ]:
vocab = processor.vocabulary

relation_tokens = sorted(
    [vocab[w] for w in ['left', 'right', 'behind', 'front'] if w in vocab])
color_tokens = sorted(
    [vocab[w] for w in ALL_POSSIBLE_COLORS if w in vocab])
#     [vocab[w] for w in ['blue', 'brown', 'cyan', 'green', 'red', 'purple', 'yellow', 'gray']])
shapes_tokens = sorted(
    [vocab[w] for w in ['cylinder', 'sphere', 'cube'] if w in vocab])
materials_tokens = sorted(
    [vocab[w] for w in ['metal', 'rubber'] if w in vocab])
size_tokens = sorted(
    [vocab[w] for w in ['small', 'large'] if w in vocab])

random_baseline = {
    'relation':  1 / len(relation_tokens),
    'color':  1 / len(color_tokens),
    'shapes':  1 / len(shapes_tokens),
    'materials':  1 / len(materials_tokens),
    'size':  1 / len(size_tokens),
    'identity':  1 / len(processor.vocabulary),
}

In [ ]:
random_baseline

In [ ]:
from torch.utils.data import Subset
import random

In [ ]:
test_dataset_original = test_dataset
systematic_dataset_original = systematic_dataset

In [ ]:
train_indices = random.sample(range(len(train_dataset)), k=500)
train_dataset_subset = Subset(train_dataset, train_indices)
test_indices = random.sample(range(len(test_dataset)), k=500)
test_dataset_subset = Subset(test_dataset, test_indices)
systematic_indices = random.sample(range(len(systematic_dataset)), k=500)
systematic_dataset_subset = Subset(systematic_dataset, systematic_indices)
common_systematic_indices = random.sample(range(len(common_systematic_dataset)), k=500)
common_systematic_dataset_subset = Subset(common_systematic_dataset, systematic_indices)


# train_dataset_subset = train_dataset
# test_dataset_subset = test_dataset
# systematic_dataset_subset = systematic_dataset

In [ ]:
%%time

collate_fns = {
    'selected': [
        ('color', CollatorForMaskedSelectedTokens(config, processor, tokens=color_tokens)),
        ('shapes', CollatorForMaskedSelectedTokens(config, processor, tokens=shapes_tokens)),
        ('materials', CollatorForMaskedSelectedTokens(config, processor, tokens=materials_tokens)),
        ('size', CollatorForMaskedSelectedTokens(config, processor, tokens=size_tokens)),
    #     ('relation', CollatorForMaskedSelectedTokens(config, processor, tokens=relation_tokens)),
    #     ('identity', IdentityCollator(config, processor)),
    ],
    'random': [
        ('color', CollatorForMaskedRandomSelectedTokens(config, processor, tokens=color_tokens, p=0.2)),
        ('shapes', CollatorForMaskedRandomSelectedTokens(config, processor, tokens=shapes_tokens, p=0.2)),
        ('materials', CollatorForMaskedRandomSelectedTokens(config, processor, tokens=materials_tokens, p=0.2)),
        ('size', CollatorForMaskedRandomSelectedTokens(config, processor, tokens=size_tokens, p=0.2)),
    #     ('relation', CollatorForMaskedRandomSelectedTokens(config, processor, tokens=relation_tokens, p=0.2)),
    #     ('identity', IdentityCollator(config, processor)),
    ]
}
dlkwargs = {
    'batch_size': 512,
    'num_workers': int(os.environ.get("SLURM_CPUS_PER_TASK", 4)),
    'pin_memory': torch.cuda.is_available(),
}

all_results = {}
for type_, fns_by_category in collate_fns.items():
    results = {}
    for name, collate_fn in fns_by_category:
        train_loader = DataLoader(train_dataset_subset, shuffle=True, collate_fn=collate_fn, **dlkwargs)
        test_loader = DataLoader(test_dataset_subset, shuffle=False, collate_fn=collate_fn, **dlkwargs)
        systematic_loader = DataLoader(systematic_dataset_subset, shuffle=False, collate_fn=collate_fn, **dlkwargs)
        common_systematic_loader = DataLoader(common_systematic_dataset_subset, shuffle=False, collate_fn=collate_fn, **dlkwargs)

        test_results = trainer.test(training_model, dataloaders=[test_loader, systematic_loader])
        raw_results = trainer.test(training_model, dataloaders=[train_loader, common_systematic_loader])
        train_results = [
            {k.replace('test_', 'train_'): v for k, v in raw_results[0].items()}]
        common_results = [
            {k.replace('systematic_test', 'common_systematic_test'): v for k, v in raw_results[1].items()}]
        results[name] = test_results + train_results + common_results

#     all_results[type_] = results

In [ ]:
all_results

In [ ]:
all_results['config'] = vars(config)
all_results['random_baseline'] = random_baseline

In [ ]:
assert False

In [ ]:
with open(f'outputs/results/{exp_name}.json', 'w') as fp:
    json.dump(all_results, fp)

In [ ]:
results = all_results['selected']
categories = results.keys()
values1 = [results[cat][0]['test_acc/dataloader_idx_0'] for cat in categories]
values2 = [results[cat][1]['systematic_test_acc/dataloader_idx_1'] for cat in categories]
values0 = [results[cat][2]['train_acc/dataloader_idx_0'] for cat in categories]
valuesR = [random_baseline[cat] for cat in categories]

plt.figure(figsize=(9,5))

# Define width of bars
bar_width = 0.2

# Create bars for each category
plt.bar([x - 1.5*bar_width for x in range(len(categories))], values0, width=bar_width, label='Train')
plt.bar([x - 0.5*bar_width for x in range(len(categories))], values1, width=bar_width, label='Test')
plt.bar([x + 0.5*bar_width for x in range(len(categories))], values2, width=bar_width, label='Systematic')
plt.bar([x + 1.5*bar_width for x in range(len(categories))], valuesR, width=bar_width, label='Random')

xs =  list(range(len(categories)))
for i, (v0, v1, v2, vr) in enumerate(zip(values0, values1,values2, valuesR)):
    plt.text(xs[i] - 1.5*bar_width, v0 + 0.01, f'{v0:.2f}', ha='center')
    plt.text(xs[i] - 0.5*bar_width, v1 + 0.01, f'{v1:.2f}', ha='center')
    plt.text(xs[i] + 0.5*bar_width, v2 + 0.01, f'{v2:.2f}', ha='center')
    plt.text(xs[i] + 1.5*bar_width, vr + 0.01, f'{vr:.2f}', ha='center')

# Add labels and title
plt.title(title_exp_name)
plt.xlabel('Measure')
plt.ylabel('Accuracy')
plt.ylim(0., 1.05)
# plt.title('Bar Chart with Two Columns Per Category')
plt.xticks(range(len(categories)), categories)
plt.legend(framealpha=0.8)

# Show plot
plt.show()

In [ ]:
results = all_results['random']
categories = results.keys()
values1 = [results[cat][0]['test_acc/dataloader_idx_0'] for cat in categories]
values2 = [results[cat][1]['systematic_test_acc/dataloader_idx_1'] for cat in categories]
values0 = [results[cat][2]['train_acc/dataloader_idx_0'] for cat in categories]
valuesR = [random_baseline[cat] for cat in categories]

plt.figure(figsize=(9,5))

# Define width of bars
bar_width = 0.2

# Create bars for each category
plt.bar([x - 1.5*bar_width for x in range(len(categories))], values0, width=bar_width, label='Train')
plt.bar([x - 0.5*bar_width for x in range(len(categories))], values1, width=bar_width, label='Test')
plt.bar([x + 0.5*bar_width for x in range(len(categories))], values2, width=bar_width, label='Systematic')
plt.bar([x + 1.5*bar_width for x in range(len(categories))], valuesR, width=bar_width, label='Random')

xs =  list(range(len(categories)))
for i, (v0, v1, v2, vr) in enumerate(zip(values0, values1,values2, valuesR)):
    plt.text(xs[i] - 1.5*bar_width, v0 + 0.01, f'{v0:.2f}', ha='center')
    plt.text(xs[i] - 0.5*bar_width, v1 + 0.01, f'{v1:.2f}', ha='center')
    plt.text(xs[i] + 0.5*bar_width, v2 + 0.01, f'{v2:.2f}', ha='center')
    plt.text(xs[i] + 1.5*bar_width, vr + 0.01, f'{vr:.2f}', ha='center')

# Add labels and title
plt.title(title_exp_name)
plt.xlabel('Measure')
plt.ylabel('Accuracy')
plt.ylim(0., 1.05)
# plt.title('Bar Chart with Two Columns Per Category')
plt.xticks(range(len(categories)), categories)
plt.legend()

# Show plot
plt.show()

In [ ]:
assert False

In [ ]:
idx = 30
question = train_dataset.questions[idx]
image_idx = question['image_index']
scene = train_dataset.indexed_scenes[image_idx]

question_str = question['question']
answer_str = question['answer']

In [ ]:
config.rels_to_sample = 0
config.only_front_right_relations = False
config.filter_symmetric_relations = True
config.display_object_properties = False

s = Scene.from_dict(scene,
                    shuffle_relations=True,
                    relations_to_sample=config.rels_to_sample,
                    only_front_right=config.only_front_right_relations,
                    filter_symmetric=config.filter_symmetric_relations,
                    always_display_properties=config.display_object_properties)

In [ ]:
config.rels_to_sample, config.only_front_right_relations, config.filter_symmetric_relations, config.display_object_properties

In [ ]:
len(s.relations), list(enumerate(s.relations))

In [ ]:
s.objects

In [ ]:
str(s)

In [ ]:
relations_to_sample = 20
only_front_right_relations = False
filter_symmetric_relations = True
display_object_properties = False

n_rels = []
n_tokens = []
for idx in range(len(train_dataset)):
    question = train_dataset.questions[idx]

    image_idx = question['image_index']
    scene = train_dataset.indexed_scenes[image_idx]

    question_str = question['question']
    answer_str = question['answer']

    s = Scene.from_dict(scene,
                        shuffle_relations=True,
                        relations_to_sample=relations_to_sample,
                        only_front_right=only_front_right_relations,
                        filter_symmetric=filter_symmetric_relations,
                        always_display_properties=display_object_properties)
    
    n_rels.append(len(s.relations))
    n_tokens.append(len(str(s).split()))

In [ ]:
print(min(n_rels), max(n_rels))
print(min(n_tokens), max(n_tokens))

In [ ]:
plt.hist(n_rels, bins=10)
plt.plot()

In [ ]:
plt.hist(n_tokens, bins=10)
plt.plot()

In [ ]:
max(n_tokens)

In [ ]:
only_front_right_relations = False
filter_symmetric_relations = True
display_object_properties = False

n_rels = []
n_tokens = []
for idx in range(len(train_dataset)):
    question = train_dataset.questions[idx]

    image_idx = question['image_index']
    scene = train_dataset.indexed_scenes[image_idx]

    question_str = question['question']
    answer_str = question['answer']
    
    s = Scene.from_dict(scene,
                        shuffle_relations=True,
                        relations_to_sample=50,
                        only_front_right=only_front_right_relations,
                        filter_symmetric=filter_symmetric_relations,
                        always_display_properties=display_object_properties)
    
    n_rels.append(len(s.relations))
    n_tokens.append(len(str(s).split()))
    

In [ ]:
print(min(n_rels), max(n_rels))
print(min(n_tokens), max(n_tokens))

In [ ]:
plt.hist(n_rels, bins=10)
plt.plot()

In [ ]:
plt.hist(n_tokens, bins=10)
plt.plot()

In [ ]:
only_front_right_relations = False
filter_symmetric_relations = True
display_object_properties = True

n_rels = []
n_tokens = []
for idx in range(len(train_dataset)):
    question = train_dataset.questions[idx]

    image_idx = question['image_index']
    scene = train_dataset.indexed_scenes[image_idx]

    question_str = question['question']
    answer_str = question['answer']

    s = Scene.from_dict(scene,
                        shuffle_relations=True,
                        relations_to_sample=50,
                        only_front_right=only_front_right_relations,
                        filter_symmetric=filter_symmetric_relations,
                        always_display_properties=display_object_properties)
    
    n_rels.append(len(s.relations))
    n_tokens.append(len(str(s).split()))
    

In [ ]:
print(min(n_rels), max(n_rels))
print(min(n_tokens), max(n_tokens))

In [ ]:
plt.hist(n_rels, bins=10)
plt.plot()

In [ ]:
plt.hist(n_tokens, bins=10)
plt.plot()

In [ ]:
dataset_path = Path('/mnt/ialabnas/datasets/CLEVR_CoGenT_v1.0')
scenes_path = dataset_path / 'scenes/CLEVR_trainA_scenes.json'
with scenes_path.open('r') as fp:
    scenes = json.load(fp)
    
questions_path = dataset_path / 'questions/CLEVR_trainA_questions.json'
with questions_path.open('r') as fp:
    questions = json.load(fp)

In [ ]:
questions['questions'][3]

In [ ]:
def n_relations(scene):
    relations = ['behind', 'front']#, 'left', 'right']
    count = 0
    relationships = scene['relationships']
    for r in relations:
        count += sum(map(len, relationships[r]))
    return count

n_relations(scenes['scenes'][0])

In [ ]:
(scenes['scenes'][0])

In [ ]:
n_relations_per_example = list(map(n_relations, scenes['scenes']))
plt.hist(n_relations_per_example)
plt.plot()

In [ ]:
def n_objects(scene):
    return len(scene['objects'])

In [ ]:
n_objects_per_example = list(map(n_objects, scenes['scenes']))
plt.hist(n_objects_per_example, bins=max(n_objects_per_example)-3)
plt.plot()

In [ ]:
def object_to_text(obj, obj_idx):
    properties = ['size', 'color', 'material', 'shape']
    obj_str = [f'[O{obj_idx}]'] + [obj[prop] for prop in properties]
    return ' '.join(obj_str)

def relations_to_text(relations):
#     relation_types = ['behind', 'front', 'left', 'right']
    relation_types = ['front', 'right'] # symetric relations

    relations_str = []
    for relation_type in relation_types:
        relation = relations[relation_type]
        for obj, subjs in enumerate(relation):
            relation_str = [f'[O{obj}] {relation_type} [O{subj}]' for subj in subjs]
            relations_str.extend(relation_str)
    
    return relations_str

def scene_to_txt(scene, rels_to_sample=None, shuffle=True):
    objs_strs = []
    for obj_idx, obj in enumerate(scene['objects']):
        objs_strs.append(object_to_text(obj, obj_idx=obj_idx))
    
    relations_strs = relations_to_text(scene['relationships'])
    if rels_to_sample and rels_to_sample < len(relations_strs):
        relations_strs = random.sample(relations_strs, k)
    if shuffle:
        random.shuffle(relations_strs)

    return ' [SEP] '.join(objs_strs + relations_strs)

In [ ]:
from PIL import Image
ip = '/mnt/ialabnas/datasets/CLEVR_CoGenT_v1.0/images/trainA/' + questions['questions'][8888]['image_filename']
Image.open(ip)

In [ ]:
object_to_text(scenes['scenes'][8888]['objects'][0], obj_idx=0)

In [ ]:
object_to_text(scenes['scenes'][8888]['objects'][1], obj_idx=1)

In [ ]:
relations_to_text(scenes['scenes'][8888]['relationships'])

In [ ]:
scene_to_txt(scenes['scenes'][8888])

In [ ]:
# scene_to_txt(scenes['scenes'][8888], rels_to_sample=5)

In [ ]:
to_str = lambda s: str(Scene.from_dict(s, 
                                        relations_to_sample=config.rels_to_sample,
                                        only_front_right=config.only_front_right_relations,
                                        filter_symmetric=config.filter_symmetric_relations,
                                        always_display_properties=config.display_object_properties))

str_len_per_example = list(map(len, map(str.split, map(to_str, scenes['scenes']))))
plt.hist(str_len_per_example)
plt.plot()

In [ ]:
str_fnc = lambda s: str(Scene.from_dict(s, relations_to_sample=50,
                                        only_front_right=config.only_front_right_relations,
                                        filter_symmetric=config.filter_symmetric_relations,
                                        always_display_properties=config.display_object_properties))

str_len_per_example = list(map(len, map(str.split, map(str_fnc, scenes['scenes']))))
plt.hist(str_len_per_example)
plt.plot()

In [ ]:
vocabulary = set()
for scene in scenes['scenes']:
    scene_text = scene_to_txt(scene)
    vocabulary.update(scene_text.split())

vocabulary = ['[CLS]', '[PAD]'] + list(sorted(vocabulary))

In [ ]:
len(vocabulary), vocabulary

## All Possible Colors

In [ ]:
all_possible_colors = set()
for n_colors in ['8c', '27c', '64c', '125c', '216c']:
    vocab_path = f'/workspace1/fidelrio/CLEVR_CoGenT_v1.0/colored-v2/{n_colors}/vocab.txt'
    with open(vocab_path) as fp:
        vocab = list(map(str.strip, fp.readlines()))
        color_vocab = [w for w in vocab if w.startswith('#')]
        all_possible_colors.update(color_vocab)

In [ ]:
len(all_possible_colors)

In [ ]:
import textwrap

print('\n'.join(textwrap.wrap(', '.join([f"'{w}'" for w in all_possible_colors]), width=103)))